# Phase stability for fluid mixtures with SAFT-$\gamma$-Mie

In this notebook, phase stability calculations for fluid mixtures with ```saftgammamie``` EoS are illustrated.

First, all the needed modules are imported.

- numpy: numerical interface and work with arrays
- sgtpy: package with SAFT-$\gamma$-Mie EoS and tpd functions.

In [1]:
import numpy as np
from sgtpy import component, mixture, saftgammamie

Now, pure components are configured and created with the ```component``` function. Then, a mixture is created with them using the ```mixture``` function or by adding (`+`) pure components. The interaction parameters are set up with the ```mixture.saftgammamie``` method. Finally, the ```eos``` object is created with the ```saftgammamie``` function.

The ```eos``` object includes all the necessary methods to compute phase equilibria and interfacial properties using SAFT-$\gamma$-Mie EoS.

For this notebook stability calculations are exemplified for a mixture of ethanol and water and for a mixture of 2-butanol and water.

In [2]:
# creating pure components
ethanol = component(GC={'CH3':1, 'CH2OH':1})
butanol2 = component(GC={'CH3':2, 'CH2':1, 'CHOH':1})
water = component(GC={'H2O':1})

# setting up mixture of ethanol and water
mix1 = mixture(ethanol, water)
# or by adding the pure components
mix1 = ethanol + water

mix1.saftgammamie()
eos1 = saftgammamie(mix1)

# setting up  mixture of 2-butanol and water
mix2 = mixture(butanol2, water)
# or by adding the pure components
mix2 = butanol2 + water

mix2.saftgammamie()
eos2 = saftgammamie(mix2)

## Tangent plane distance function

This function measures the relative stability of a phase against another. To study the stability with this criteria one must found a minimum of the tpd function and then check the sign of the distance. There are three scenarios:
- tpd > 0: the phase is stable
- tpd = 0: the phase is in equilibrium with the trial phase.
- tpd < 0: the phase is not stable. The addition of the trial phase to the system would decrease the Gibbs energy of the system.

To perform one single minimization you should use the ```tpd_min``` function. This function requires an initial guess for the trial phase composition (w0), the global composition (z), temperature (K), pressure (Pa) and aggregation state of the trial phase (stateW) and mixture (stateZ) and it returns the composition of the minimum and its tpd value.

In [3]:
from sgtpy.equilibrium import tpd_min

# tpd function for mixture of ethanol and water
T = 350.  # K
P = 7e4   # Pa
z = np.array([0.3, 0.7])

# initial guess for the composition of the trial phase
w0 = np.array([0.5, 0.5])
tpd_min(w0, z, T, P, eos1, stateW = 'V', stateZ = 'L')

(array([0.58031872, 0.41968128]), -0.1506502231574185)

It can be noted that at the given global composition, temperature and pressure, the liquid mixture will split to create vapor phase. The composition obtained with the minimization can be used as an initial guess for a flash calculation.

Similarly, you could check the stability of a vapor mixture under the same conditions.

In [4]:
# tpd function for mixture of ethanol and water
T = 350.  # K
P = 7e4   # Pa
z = np.array([0.3, 0.7])

# initial guess for the composition of the trial phase
w0 = np.array([0.1, 0.9])
tpd_min(w0, z, T, P, eos1, stateW = 'L', stateZ = 'V')

(array([0.05447079, 0.94552921]), -0.23646264561186037)

Again, it is noted that the vapor phase would split to form a liquid phase. The composition of the found minimum can be used as an initial guess for a flash calculation.

The same strategy could be used for a mixture of 2-butanol and water.

In [5]:
T = 350.  # K
P = 1e5   # Pa
z = np.array([0.2, 0.8])

# initial guess for the composition of the trial phase
w0 = np.array([0.01, 0.99])
tpd_min(w0, z, T, P, eos2, stateW = 'V', stateZ = 'L')

(array([0.38103264, 0.61896736]), 0.3265256251158346)

As tpd > 0, It can be seen that for this mixture at these conditions no vapor phase will be formed.

However, the stability of the mixture could be studied against another liquid phase.

In [6]:
# initial guess for the composition of the trial phase
w0 = np.array([0.1, 0.9])
tpd_min(w0, z, T, P, eos2, stateW = 'L', stateZ = 'L')

(array([0.03974618, 0.96025382]), -0.013397063751130256)

In [7]:
# initial guess for the composition of the trial phase
w0 = np.array([0.5, 0.5])
tpd_min(w0, z, T, P, eos2, stateW = 'L', stateZ = 'L')

(array([0.45608682, 0.54391318]), -0.011966954245883343)

For the mixture of 2-butanol and water, two liquid phase minima have been found with negative tpd. This implies that the mixture will split into two liquids (LLE).

The compositions of the found minima can be used to compute the phase equilibria.

You can also try to find all the minima of a mixture with one single function. In this case, the function ```tpd_minimas``` will try to find ```nmin``` minima. This function returns the composition of the minima and an array with their tpd values.

In [8]:
from sgtpy.equilibrium import tpd_minimas

nmin = 3
tpd_minimas(nmin, z, T, P, eos2, 'L', 'L')

((array([0.95158339, 0.04841661]),
  array([0.03974559, 0.96025441]),
  array([0.45608421, 0.54391579])),
 array([ 0.28719629, -0.01339706, -0.01196695]))

Similarly, the function ```lle_init``` can be used to generate initial guesses for LLE calculation.

In [9]:
from sgtpy.equilibrium import lle_init

lle_init(z, T, P, eos2)

(array([0.95158339, 0.04841661]), array([0.03974559, 0.96025441]))

For further information of any of these functions just run: ```function?```